# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 10


API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://imedhub.io/")
ed.links

['tel: +91 7075532955',
 'tel: +91 7075532955',
 'javascript:void(0);',
 'javascript:void(0);',
 'javascript:void(0);',
 'javascript:void(0);',
 'javascript:void(0);',
 'index.php',
 'index.php',
 'javascript:void(0);',
 'healthsystem.php',
 'second_opinion.php',
 'pricing.php',
 'blog.php',
 'contact.php',
 'healthsystem.php',
 'health_facilities.php',
 'patient_portal.php',
 '#dc-bannerslider',
 '#dc-bannerslider',
 'javascript:void(0);',
 'javascript:void(0);',
 'javascript:void(0);',
 'javascript:void(0);',
 'javascript:void(0);',
 'javascript:void(0);',
 'blog_post.php?id=34',
 'blog_post.php?id=36',
 'blog_post.php?id=38',
 'blog_post.php?id=40',
 'blog_post.php?id=42',
 'blog_post.php?id=32',
 'blog_post.php?id=44',
 'https://play.google.com/store/apps/details?id=com.imedworks.imed_health',
 'https://apps.apple.com/us/app/imedhub/id6472778505',
 'blog_post.php?id=28',
 'blog_post.php?id=28',
 'blog_post.php?id=23',
 'blog_post.php?id=23',
 'blog_post.php?id=12',
 'blog_post.php?

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://imedhub.io/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
tel: +91 7075532955
tel: +91 7075532955
javascript:void(0);
javascript:void(0);
javascript:void(0);
javascript:void(0);
javascript:void(0);
index.php
index.php
javascript:void(0);
healthsystem.php
second_opinion.php
pricing.php
blog.php
contact.php
healthsystem.php
health_facilities.php
patient_portal.php
#dc-bannerslider
#dc-bannerslider
javascript:void(0);
javascript:void(0);
javascript:void(0);
javascript:void(0);
javascript:void(0);
javascript:void(0);
blog_post.php?id=34
blog_post.php?id=36
blog_post.php?id=38
blog_post.php?id=40
blog_post.php?id=42
blog_post.php?id=32
blog_post.php?id=44
https://play.google.com/store/apps/details?id=com.imedworks.imed_health
https://apps.apple.com/us/app/imed

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 'blog/inference-providers-nebius-novita-hyperbolic',
 '/spaces',
 '/models',
 '/perplexity-ai/r1-1776',
 '/deepseek-ai/DeepSeek-R1',
 '/microsoft/OmniParser-v2.0',
 '/stepfun-ai/Step-Audio-Chat',
 '/stepfun-ai/stepvideo-t2v',
 '/models',
 '/spaces/nanotron/ultrascale-playbook',
 '/spaces/lllyasviel/LuminaBrush',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/microsoft/OmniParser-v2',
 '/spaces/m-ric/open_Deep-Research',
 '/spaces',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/facebook/natural_reasoning',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://imedhub.io/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://imedhub.io/about.php'}, {'type': 'contact page', 'url': 'https://imedhub.io/contact.php'}, {'type': 'blog page', 'url': 'https://imedhub.io/blog.php'}]}
Landing page:
Webpage Title:
iMedHub - VitalBuddy
Webpage Contents:
Remote Healthcare Monitoring
India: +91 8885303042
USA: +1 (866)8860122
Home
Services
Health Monitoring System
Second Opinion
Pricing
Blog
Contact
Book Demo
Empowering Long Distance
Healthcare Monitoring
empower Non-Resident Indians (NRIs) to monitor their elderly parents’ health remotely
Learn More
Book Demo
Get Your
Medical Second Opinion
FROM
USA BOARD-CERTIFIED
PHYSICIANS
View Hospitals
Know More
PR
EV
Previous
NE
XT
Next
Comprehensive FREE health screening
every 3 Months!
The Free Lab Test package is a comprehensive health screening test that evaluates key health indicators essential for overall wellness.  
										Once the tests are complete, the lab reports will be uploaded to the iMedHub mobile ap

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("imedworkshub", "https://imedhub.io/")

Found links: {'links': [{'type': 'about page', 'url': 'https://imedhub.io/about.php'}, {'type': 'contact page', 'url': 'https://imedhub.io/contact.php'}]}


"You are looking at a company called: imedworkshub\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\niMedHub - VitalBuddy\nWebpage Contents:\nRemote Healthcare Monitoring\nIndia: +91 8885303042\nUSA: +1 (866)8860122\nHome\nServices\nHealth Monitoring System\nSecond Opinion\nPricing\nBlog\nContact\nBook Demo\nEmpowering Long Distance\nHealthcare Monitoring\nempower Non-Resident Indians (NRIs) to monitor their elderly parents’ health remotely\nLearn More\nBook Demo\nGet Your\nMedical Second Opinion\nFROM\nUSA BOARD-CERTIFIED\nPHYSICIANS\nView Hospitals\nKnow More\nPR\nEV\nPrevious\nNE\nXT\nNext\nComprehensive FREE health screening\nevery 3 Months!\nThe Free Lab Test package is a comprehensive health screening test that evaluates key health indicators essential for overall wellness.  \n\t\t\t\t\t\t\t\t\t\tOnce the tests are complete, the lab reports will be uploade

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("iMedHubIO", "https://imedhub.io/")

Found links: {'links': [{'type': 'about page', 'url': 'https://imedhub.io/about.php'}, {'type': 'contact page', 'url': 'https://imedhub.io/contact.php'}, {'type': 'blog page', 'url': 'https://imedhub.io/blog.php'}, {'type': 'patient portal page', 'url': 'https://imedhub.io/patient_portal.php'}, {'type': 'health system page', 'url': 'https://imedhub.io/healthsystem.php'}, {'type': 'health facilities page', 'url': 'https://imedhub.io/health_facilities.php'}, {'type': 'second opinion page', 'url': 'https://imedhub.io/second_opinion.php'}, {'type': 'pricing page', 'url': 'https://imedhub.io/pricing.php'}]}


```markdown
# iMedHubIO: Redefining Healthcare Monitoring for Seniors

## About Us
iMedHubIO specializes in **Remote Healthcare Monitoring**, empowering Non-Resident Indians (NRIs) to monitor the health of their elderly parents from a distance. Our innovative solutions leverage cutting-edge technology to provide comprehensive health monitoring services and facilitate second opinions from board-certified physicians in the USA.

## Our Services
- **Health Monitoring System**: Seamlessly track the health metrics of your loved ones with real-time data from connected smart devices (like Apple Watch, Fitbit, etc.).
- **Emergency Alerts**: Our robust monitoring system sends instant alerts during emergencies, ensuring timely interventions by our clinical team.
- **Second Opinions**: Receive expert medical consultations from qualified physicians, enhancing decision-making regarding health management.

### Key Features
- **Comprehensive Free Health Screening**: Every three months, we offer an exhaustive health screening package that evaluates key indicators, ensuring well-being is continuously monitored.
- **Dedicated Clinical Monitoring Team**: Our team of licensed nurses and trained medical assistants provide proactive and compassionate care, available 24/7/365 globally.
- **Multilingual Support**: We offer health services in English and various regional languages to cater to diverse populations.

## Target Customers
At iMedHubIO, we primarily serve Non-Resident Indians who wish to keep a close watch on their elderly parents’ health from afar. Our platform is designed for families who want to ensure their loved ones maintain their independence while receiving necessary medical support, regardless of geographical barriers.

## Company Culture
We believe in a culture centered around **compassionate care** and **technological innovation**. Our mission is to ensure every senior enjoys a fulfilling life, supported by the latest advancements in telehealth and remote monitoring. We foster an environment that values teamwork, encourages learning, and emphasizes the importance of patient-centered care.

## Join Us
We are always looking for passionate and dedicated individuals to join our dynamic team. Careers at iMedHubIO offer the chance to be part of a movement that is reinventing elderly care through technology. If you're interested in making a tangible difference in people's lives, explore our careers page for available opportunities.

## Get in Touch
For more information or to book a demo, contact us:
- **India**: +91 8885303042
- **USA**: +1 (866) 8860122

Stay connected, stay informed, and let iMedHubIO empower you to care for your loved ones, regardless of distance!
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [1]:
stream_brochure("iMedHubIO", "https://cnn.com/")
gr.Interface(fn=stream_brochure("iMedHubIO", "https://cnn.com/")
, inputs="textbox", outputs="textbox").launch()

NameError: name 'stream_brochure' is not defined

In [26]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("iMedHubIO", "https://cnn.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.cnn.com/about'}, {'type': 'careers page', 'url': 'https://careers.wbd.com/cnnjobs'}]}


# iMedHubIO Company Brochure

## About Us
iMedHubIO is a cutting-edge technology solutions provider dedicated to transforming the healthcare landscape through innovative digital solutions. Our focus is on enhancing patient care, streamlining medical processes, and utilizing data analytics to improve health outcomes.

## Company Culture
At iMedHubIO, we foster a collaborative and inclusive work environment where creativity and innovation thrive. Our team is passionate about harnessing technology to solve real-world problems in healthcare. We celebrate diversity, encourage professional development, and believe that every employee is key to our mission of improving health outcomes.

### Core Values:
- **Integrity**: We act with honesty and transparency.
- **Innovation**: We embrace creativity and new ideas.
- **Collaboration**: We work together to achieve shared goals.
- **Empathy**: We understand and prioritize the needs of patients and healthcare providers.

## Our Customers
iMedHubIO is proud to serve a diverse array of clients, including:
- Hospitals and healthcare providers
- Research institutions
- Health tech startups

Our solutions are designed to meet the unique needs of each customer, ensuring that they can provide superior care and streamline their operations.

## Careers at iMedHubIO
We are always on the lookout for talented and driven individuals to join our team. Whether you’re a recent graduate or an experienced professional, at iMedHubIO, you can pursue a fulfilling career while making a positive impact in the healthcare sector.

### Current Openings:
- Software Engineers
- Data Analysts
- Healthcare Consultants
- Project Managers

We offer competitive salaries, flexible work arrangements, and opportunities for continuous learning and advancement.

## Join Us 
If you are passionate about transforming healthcare through technology, we invite you to explore career opportunities with us. Together, we can make a difference in the lives of countless patients and healthcare practitioners.

### Contact Us
For more information about our solutions, culture, or careers, visit our website at [iMedHubIO.com](http://imedhubio.com).

---

At iMedHubIO, innovation meets compassion. Join us on our mission to enhance healthcare experiences for everyone.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>